In [1]:
using Distributed

addprocs(10)

@everywhere begin
    
    using CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions
        
    #Necessary functions

    function alpha_gt(x::Float64, y::Float64, omega::Float64, t::Float64)
    
        al=0
        if(abs(x-y)<t) 
            
             al=exp(-((x-y)^2)/(omega^2))

        end
        return al
    end
    
    function alpha_tri(x::Float64, y::Float64, slope::Float64, t::Float64)
        
        alpha=0.0
        
        if ( abs(x-y) < t) 
            
            alpha=1- (slope*abs(x-y))
            
        end
            
        return max(0,alpha)
        
    end
    
    function mnnds(a::Vector{Float64})
    
        a=filter!(!isnan,a)
        if(length(a)>1)
            a=sort(a,rev=true)
            nnd=zeros(Float64,length(a))
            nnd[1]=a[2]-a[1]
            nnd[length(nnd)]=a[length(a)]-a[length(a)-1]
            for i in 2:((length(a)-1))
                nnd[i]=min((a[i-1]-a[i]),(a[i]-a[i+1]))
            end
            ranges=a[1]-a[end]
            mnnd=sum(nnd)/length(nnd)
            mmax=ranges/(length(a)-1)

            return mnnd/mmax

            else return 0.0
            end
        
    end
    
    
    function qgprob(n::Int64)
    
        #All possible phenotypes
        pheno= collect(1:(2*n+1)) ./ (2*n+1)
        nt=length(pheno)
    
        G=zeros(Float64,n+1,n+1,n+1)
    
        for i in 0:n, j in 0:i, k in max(0,(i+j-n)):min(n,(i+j))
                    m=collect(0:min(j,k,i+j-k))
                    G[1+i,1+j,1+k]=sum(pdf.(Hypergeometric(i,n-i,j),m).*pdf.(Binomial.(i+j .- (2 .* m)),k .- m))
        end
    
        for k in 0:n
            G[:,:,1+k]=G[:,:,1+k]+transpose(G[:,:,1+k])
            for i1 in 0:n
                G[i1+1,i1+1,k+1] /= 2
            end
        end
    
        ind_haplR=zeros(Float64,2*n+1, 2*n+1)
    
        for k in 0:n
            for i in 0:n
                 ind_haplR[1+i,1+k] = G[1+i,1,1+k]
                for j in 0:n
                    ind_haplR[1+j+n,1+k]=G[1+n,1+j,1+k]
                end
            end
        end
    
        R=zeros(Float64,nt,nt,nt)
    
        for i in 0:(2*n), j in 0:(2*n), q in 0:(2*n)
             R[1+i,1+j,1+q]= sum(ind_haplR[1+i,1 .+ (0:q)] .*
                                 ind_haplR[1+j,1+q .- (0:q)])
        end
    
        return R
    end
    
    function single_sim1(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
            nsp=size(Ng0)[1]
            nt=size(Ng0)[2]
            
            Np0=Ng0 .*Npop
            Ngen=deepcopy(Ng0)
            Np=deepcopy(Np0)
        
            dat=zeros(Float64,time+1,nsp,nt)
    
            dat[1,:,:]=Np
            
            #Start the simulation
            for m in 2:(time+1)
        
                #Determine the extinct species
                Np[findall(sum(Np,dims=2) .< 10),:] .= 0
                Ngen[findall(sum(Np,dims=2) .==0),:] .= 0
    
                if all(sum(Np,dims=2) ==0) 
                    break
                else
    
                    newgen=zeros(Float64,nsp,nt)
    
                    #Reproduction event
                    for i in findall(!iszero,sum(eachcol(Ngen)))
    
                        probs=Ngen[i,:]*Ngen[i,:]'
                    
                        newgen[i,:]=[sum(probs.*R[:,:,x1]) for x1 in 1:nt]
                    
                        newgen[i,:] ./= sum(newgen[i,:])
                        
                    end
    
                    newgen .*= sum(Np,dims=2)
    
                    #Selection event
                
                    if size(A)[1]>1
    
                        for i1 in 1:size(newgen)[1]
                    
                            #This is where we have to make big choices!! a)Whether or not to make conspecific individuals compete the 
                            #same way as heterospecifics. b) Interprete the meaning of carrying capacity. Do different numbers
                            #apply for different species? If yes, what is the interpretation of this type of competition?
                           
                            #Reduction in growth rate due to intraspecific competition (not trait-dependent!). The carrying capacity
                            #here is assumed to be the same for all species.

                            rdash=r[i1] * (1 -(sum(newgen[i1,:])/K2))
                        
                            #Impact of interspecific competition

                            comps=[(a1.*sum((A[x2,:]) .* newgen[1:end .!=i1,:]')) for x2 in 1:nt]
                        
                            Np[i1,:] = newgen[i1,:] + (newgen[i1,:] .* rdash .* (1 .-(comps ./K1)))
                        
                        end
                    
                    end
    
                    Np[findall(Np .<1)] .= 0
                    Ngen= Np ./ sum(Np,dims=2)
                    Ngen[isnan.(Ngen)].=0
                end
                
            dat[m,:,:]=Np
                
            end
        
        return dat
        
    end


    #Blanket function to create trait and population trajectories
    function getsum(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
        res=single_sim1(time,r,K1,K2,a1,A,R,Ng0,Npop)
        
        tstep=size(res)[1]
        nsp=size(res)[2]
        nt=size(res)[3]
        geno=collect(range(-1,stop=1,length=nt))
        

        pops=zeros(Float64,tstep,nsp)
        trmeans=zeros(Float64,tstep,nsp)

        for i in 1:nsp

        pops[:,i]=[sum(res[x,i,:]) for x in 1:tstep]
        trmeans[:,i]=[sum(res[x,i,:] .* geno)/sum(res[x,i,:])  for x in 1:tstep]

        end

        return([pops,trmeans])
        
    end
        
    reps=collect(1:30)
    loci=[3,4,5,6,7,8,10,15,20,50]
        
    nsp=20

    pars=collect(Iterators.product(reps,loci))
            
    function compsim(par::Tuple{Int64, Int64})
            
        rep=par[1]
        n=par[2]
            
        geno= collect(range(-1,stop=1,length=2*n+1))
        nt=length(geno)
        
        #Precalculate R, the outcomes of mating between each of pair of phenotypes
        R=qgprob(n)

        #Parameters related to the competition kernels
        omega=1.0
        slope=0.75
        t=0.1
        a1s=[0.1,0.25,0.5]
        kernel=["Gaussian","Triangle"]
        traits=["Gaussian","Uniform"]
        K1s=[1000.0,5000.0,10000.0]
        K2=5000.0
            
        res=DataFrame()
        
        for p1 in a1s, p2 in kernel, p3 in K1s, p4 in traits
                
                a1=p1
                K1=p3
                
                 #Pre-calculate coefficients of competition between pairs of phenotypes

                A=zeros(Float64,nt,nt)

                for i1 in 1:nt, i2 in 1:nt
                    
                    if p2=="Gaussian"
                        
                        A[i1,i2]=alpha_gt(geno[i1],geno[i2],omega,t)
                        
                    elseif p2=="Triangle"
                        
                        A[i1,i2]=alpha_tri(geno[i1],geno[i2],slope,t)
                    end
                end
                    
                    
                #Demographic parameters
                #intrinsic growth rate, assumed to be specific to each species,not the phenotypes.
                r=abs.(rand(Uniform(0.2,0.3),nsp))            
               
                #Starting populations
                
                N0=zeros(Float64,nsp,nt)
                rands=rand(Uniform(-0.6,0.6),nsp)

                if p4=="Gaussian"
            
                    [N0[i,:]=pdf.(truncated(Normal(rands[i],0.2),-1.0,1.0),geno) for i in 1:nsp]
                        
                    elseif p4=="Uniform"
            
                        [N0[i,:]=pdf.(Uniform(rands[i]-0.4,rands[i]+0.4),geno) for i in 1:nsp]
                end
                
                N0=N0 ./ sum(N0,dims=2)
        
        
                res1=getsum(2000,r,K1,K2,0.1,A,R,N0,1000.0)
                trdat=res1[2]
                mnnddat=[mnnds(trdat[x,:]) for x in 1:2001]
                

                append!(res,DataFrame(nloci=n,reps=rep,a1s=a1,K1s=K1,kernel=p2,traits=p4,time=1:2001,mnnd=mnnddat))

        end
        
        CSV.write(string("compdat_",rep,"_",n,"_",".csv"),res)
    end
                
end        
              

In [2]:
pmap(compsim,pars)

Worker 2 terminated.
Unhandled Task ERROR: IOError: read: connection reset by peer (ECONNRESET)
Stacktrace:
  [1] wait_readnb(x::Sockets.TCPSocket, nb::Int64)
    @ Base .\stream.jl:410
  [2] (::Base.var"#wait_locked#715")(s::Sockets.TCPSocket, buf::IOBuffer, nb::Int64)
    @ Base .\stream.jl:949
  [3] unsafe_read(s::Sockets.TCPSocket, p::Ptr{UInt8}, nb::UInt64)
    @ Base .\stream.jl:955
  [4] unsafe_read
    @ .\io.jl:761 [inlined]
  [5] unsafe_read(s::Sockets.TCPSocket, p::Base.RefValue{NTuple{4, Int64}}, n::Int64)
    @ Base .\io.jl:760
  [6] read!
    @ .\io.jl:762 [inlined]
  [7] deserialize_hdr_raw
    @ C:\Users\mihir\AppData\Local\Programs\Julia-1.9.2\share\julia\stdlib\v1.9\Distributed\src\messages.jl:167 [inlined]
  [8] message_handler_loop(r_stream::Sockets.TCPSocket, w_stream::Sockets.TCPSocket, incoming::Bool)
    @ Distributed C:\Users\mihir\AppData\Local\Programs\Julia-1.9.2\share\julia\stdlib\v1.9\Distributed\src\process_messages.jl:172
  [9] process_tcp_streams(r_stre

LoadError: ProcessExitedException(2)
Stacktrace:
  [1] [0m[1mtry_yieldto[22m[0m[1m([22m[90mundo[39m::[0mtypeof(Base.ensure_rescheduled)[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m.\[39m[90m[4mtask.jl:920[24m[39m
  [2] [0m[1mwait[22m[0m[1m([22m[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m.\[39m[90m[4mtask.jl:984[24m[39m
  [3] [0m[1mwait[22m[0m[1m([22m[90mc[39m::[0mBase.GenericCondition[90m{ReentrantLock}[39m; [90mfirst[39m::[0mBool[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m.\[39m[90m[4mcondition.jl:130[24m[39m
  [4] [0m[1mwait[22m
[90m    @[39m [90m.\[39m[90m[4mcondition.jl:125[24m[39m[90m [inlined][39m
  [5] [0m[1mtake_buffered[22m[0m[1m([22m[90mc[39m::[0mChannel[90m{Any}[39m[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m.\[39m[90m[4mchannels.jl:457[24m[39m
  [6] [0m[1mtake![22m[0m[1m([22m[90mc[39m::[0mChannel[90m{Any}[39m[0m[1m)[22m
[90m    @[39m [90mBase[39m [90m.\[39

In [13]:
files=filter(x->contains(x,"compdat"),readdir())
    
dat=CSV.read(files[1],DataFrame)

Row,nloci,reps,a1s,K1s,kernel,traits,time,mnnd
,Int64,Int64,Float64,Float64,String15,String15,Int64,Float64
1,10,10,0.1,1000.0,Gaussian,Gaussian,1,0.432159
2,10,10,0.1,1000.0,Gaussian,Gaussian,2,0.430622
3,10,10,0.1,1000.0,Gaussian,Gaussian,3,0.429956
4,10,10,0.1,1000.0,Gaussian,Gaussian,4,0.428332
5,10,10,0.1,1000.0,Gaussian,Gaussian,5,0.42638
6,10,10,0.1,1000.0,Gaussian,Gaussian,6,0.425481
7,10,10,0.1,1000.0,Gaussian,Gaussian,7,0.423732
8,10,10,0.1,1000.0,Gaussian,Gaussian,8,0.421612
9,10,10,0.1,1000.0,Gaussian,Gaussian,9,0.41972


In [22]:
#Data analysis

using Plots,CSV, DataFrames, Random, LinearAlgebra, Distances, Distributions, SpecialFunctions

files=filter(x->contains(x,"compdat"),readdir())
    
dat=DataFrame()

for i in 1:length(files)
    
    dat=append!(CSV.read(files[i],DataFrame),dat)
    
end



In [23]:
dat

Row,nloci,reps,a1s,K1s,kernel,traits,time,mnnd
,Int64,Int64,Float64,Float64,String15,String15,Int64,Float64
1,8,9,0.1,1000.0,Gaussian,Gaussian,1,0.46993
2,8,9,0.1,1000.0,Gaussian,Gaussian,2,0.469772
3,8,9,0.1,1000.0,Gaussian,Gaussian,3,0.469333
4,8,9,0.1,1000.0,Gaussian,Gaussian,4,0.47044
5,8,9,0.1,1000.0,Gaussian,Gaussian,5,0.470225
6,8,9,0.1,1000.0,Gaussian,Gaussian,6,0.470077
7,8,9,0.1,1000.0,Gaussian,Gaussian,7,0.470634
8,8,9,0.1,1000.0,Gaussian,Gaussian,8,0.470126
9,8,9,0.1,1000.0,Gaussian,Gaussian,9,0.469607
